# unification modulo egraphs
We can recurse down and choose to match subterms by either ematching or syntactic unification

Or we can go bottom up

Why?
E-KB is interesting.
Side car egraph in a prolog might be interesting.


You might think you can eagerly take

f(t) = f(s) ---> t = s
But you can't

consider an egraph f(a) = f(b)

and now we want to solve f(a) = f(b). We process it via the unification head rule to 


https://www.philipzucker.com/unify/
![](https://www.philipzucker.com/assets/traat/unify_rules.png)

```
{t = s, S}  --->  sigma S ematch
```

 

In [ ]:
from kdrag.all import *

Egraph = dict[int, smt.ExprRef]

def find(egraph, expr):
    while expr.get_id() in egraph:
        expr = egraph[expr.get_id()]
    return expr

def union(egraph, e1, e2):
    e1 = find(egraph, e1)
    e2 = find(egraph, e2)
    if e1.eq(e2):
        return e1
    egraph[e1.get_id()] = e2
    return e2

def rebuild(egraph):
    # Rebuild the egraph
    new_egraph = {}
    for expr in egraph.values():
        
    return new_egraph

def 


In [ ]:
from kdrag.all import *

Egraph = dict[smt.ExprRef, smt.ExprRef]

def find(egraph, expr):
    while expr in egraph:
        expr = egraph[expr]
    return expr

def union(egraph, e1, e2):
    e1 = find(egraph, e1)
    e2 = find(egraph, e2)
    if e1.eq(e2):
        return e1
    egraph[e1] = e2
    return e2

def rebuild(egraph): # hmmm.
    new_egraph = {}
    for expr in egraph.keys():
        new_expr = expr.decl()(*(find(egraph, c) for c in expr.children()))
        if new_expr.eq(expr):
            new_egraph[new_expr] = find(egraph, expr)
    return new_egraph

def rebuild(egraph):
    todo = set(egraph.keys()) | set(egraph.values())
    while todo:
        e = todo.pop()
        new_expr = e.decl()(*(find(egraph, c) for c in e.children()))
        union(egraph, new_expr, e)
    return egraph

egg = {}

a,b,c = smt.Ints('a b c')
union(egg, a, b)
union(egg, b, c)
find(egg, a)
rebuild(egg)

f = smt.Function('f', smt.IntSort(), smt.IntSort())
egg = {}
union(egg, f(a), a)
union(egg, a , f(f(f(a))))
rebuild(egg)
rebuild(egg)
rebuild(egg)

def enorm(egraph, t):
    return find(egraph, t.decl()(*[enorm(c) for c in t.children()]))



def rec_add(termbank, t):
    if t not in termbank:
        termbank.add(t)
        for c in t.children():
            rec_add(termbank, c)

def compress_termbank(termbank):
    tbank = {}
    for t in termbank:
        rec_add(tbank, enorm(egraph, t))
    return tbank   

def rebuild(egraph, termbank):
    for t in termbank:
        new_expr = t.decl()(*(find(egraph, c) for c in t.children()))
        union(egraph, new_expr, t)


{f(a): a, a: f(f(f(a))), f(f(f(f(a)))): f(f(f(a)))}

In [ ]:
import kdrag.theories.int as int_

rules = [
    int_.add_assoc,
    int_.add_comm,
    int_.add_zero,
    int_.mul_assoc,
    int_.mul_comm,
    int_.mul_zero,
]
rules = [rw.rewrite_of_expr(r) for r in rules]
termbank = { a + a + a}
egraph = {}

def ematch(egraph, termbank, vs, t):
    res = []
    for ts in itertools.product(termbank, (vs)):
        subst = list(zip(vs, ts))
        t1 = enorm(egraph, smt.substitute(t, *subst))
        if t1 in termbank:
            res.append((subst, t1))
    return res

def erewrite1(egraph, termbank, rules):
    for r in rules:
        for subst, lhs in ematch(egraph, termbank, r.vs, r.lhs):
            rhs = enorm(egraph, smt.substitute(r.rhs, *subst))
            termbank.add(rhs) # recursive?
            union(egraph, lhs, rhs)





NameError: name 'itertools' is not defined

In [ ]:
def

In [ ]:
import itertools
def esolve(egraph, termbank, vs, t, s):
    res = []
    for ts in itertools.product(termbank, (vs)):
        subst = list(zip(vs, ts))
        t1 = smt.substitute(t, *subst)
        s1 = smt.substitute(s, *subst)
        if enorm(t1).eq(enorm(s1)):
            res.append(subst)
    return res

def ematch(egraph, termbank, vs, t):
    res = []
    for ts in itertools.product(termbank, (vs)):
        subst = list(zip(vs, ts))
        t1 = enorm(egraph, smt.substitute(t, *subst))
        if t1 in termbank:
            res.append((subst, t1))
    return res





Wow, claude is pretty good.


# sympy eunify
Unification is Equation Solving.


Go ground. Go flat
 multiset equations


Nearly flat lambda unification and slotted egraphs.


p == q ---->  [  othereqs ]

Cody and Max are kind of into KB ~ e-unify.

Wheels within wheels. We can have a KB running inside of e-unify to solve steps modulo a theory
But also we could have e-unify running inside KB to do narrowing / overlap modulo equations.
Could I actually make an implementation that reflects this tower?


subsumption and inductionless induction. What "counts" as being subsumed and implicit. Could have a theorem prover called at every resolution step.
Another example of wheels within wheels.

Syntactic unification is a nice base case.


In [ ]:
from dataclasses import dataclass
import itertools 
from typing import Optional

@dataclass
class MultiSet():
    elmts : list # sorted list
    def __init__(self, elts):
        self.elmts = sorted(elts)
    def solve(self, other):
        assert isinstance(other, MultiSet)
        if len(self.elmts) != len(other.elmts):
            return
        for p in itertools.permutations(other.elmts):
            yield zip(self.elmts, p)

@dataclass
def Var():
    # I'm mixing my styles here. This is a mutational style. Whatev.
    val : Optional[object]
    def __init__(self):
        pass
    def solve(self, other):
        if v.val is None:
            self.val = other
            yield []
        yield [(self, other)]

@dataclass
class Lit():
    val : object
    def __init__(self, val):
        self.val = val
    def solve(self, other):
        assert isinstance(other, Lit)
        if self.val == other.val:
            yield [(self.val, other.val)]
        else:
            return

class MySet():
    elmts : list
    def __init__(self, elts):
        self.elmts = sorted(elts)
    def solve(self, other):
        pass #allow repeats

list(MultiSet([1,2,3]).solve(MultiSet([3,2,1]))



SyntaxError: incomplete input (2314640992.py, line 28)

In [7]:
from sympy import *

f = Function('f')
x,y = symbols('x y')
solveset(f(x) - f(y), x)
solve(f(x) - f(y), x)

[y]

Substitution intersection also requires unification / is unification.
Non threaded / bottom up unification needs to merge branches.



Threads:



unitary finitary inifintary and type 0. how big set of miniaml unifiers is
elemenary, with constants and general. Whether there are extra symnbols not in E.
X1 + X2 + X3 = X4
X1 + a = X4
X1 + f(X2) = X4 

single vs multiple equations.


KB + eunify.

```
------------------------
kbE, kbR |- euP, euE, euS  ===> 
```



categoruy theory: find best sig. equalizer.
f . sig = g . sig
e-unification will be the same in a lawvere theory

https://www.cs.bu.edu/fac/snyder/publications/UnifChapter.pdf chapter from handbook
huh. wayne snyder is in boston
[Jouannaud and Kirchner 1991, 
Baader and Siekmann 1994 https://dl.acm.org/doi/abs/10.5555/185705.185711

approximate unification - we can minsat the multiequations. Or consider Least squares solutions as an anlogy. Put some kind of metric on how badly an equation is viotlated rather that descirtee 0 1. maybe go neural. neural logic pgroamming. neural unification


Bottom up eunification - Guess over a database what you'll unify on.

E-unification and narrowing
HO pattern unify
Full HO unify

I've said that higher order != has syntactic lambdas. Higher order unification could maybe seen as E-unify modulo combinator equations?

Unification modulo egraphs


boolean unification

Rational tree unification. Non well founded equations. Rationals themselves as coalgebra?

most general unifier as a greatest fixed point? Greatest something.

Categorical perspectives on substitutions.
Arrows. sure. between what? Variable sets? Term sets? Something else?


Guassian elimination means two things.
Add rows. - This is more like knuth bendix. ForAll uvars and bottom up, forward chaining
isolate variable and subst. More like unify. Exists uvars and top down, back chaining


equation breakdown. We could keep the old equations in a saturation process.
sin(x) = y --> x = arcsin(y)



Unify with branching as clauses?


https://www.cs.cmu.edu/Groups/fox/people/fp/papers/ppcp93.pdf  Higher-Order Logic Programming as Constraint Logic Programming
flex flex pairs as constraints


e-unify via ATP
factoring rules do the analog of solving unification (?) in a prolog
```
cnf(axiom, plus(X,Y) = plus(Y,X)). % The E-ness
cnf(conjecture, ~(f(X) = g(Y))).
```

ATP already unifies E-unification with knuth bendix
facotring rules
```
t != s | u = v
----------------
   s[u] = s[v]

```


Traat chapter uses something akin to multiset knuth bendix for Ac-unify

higher order unification vs higher order atp vs lambda prolog


As awlays 


I was suggesting in co-egraphs

unification modulo derivatives?
That plotkin thing
integrals?
trig?




In [ ]:
def comm_unify(vs,t1, t2):
    # todo quee is not in DNF. An and of ors...
    # Hmm. Not sufficient.
    # ok so an or of and ands
    todo = [[(t1,t2)]]
    if is_add(t):
        if is_add(s):
            if len(t.children()) = len(s.children())
                # The chilrren choices are correlated here.
                # ok, so we do need to
                # probably here we should delay this unification as long as possible
                todo = [todo.copy() + [(t[i], s[perm[i]])  for i in range(n) ] for perm in permutations(n)]
                


# unification arranged as a given clause algorithm
def given_clause_unify():


Ematch
As always, we should consider ematching first.
Here again we get a flavor of unification's complication anyway and having a global queue to look at is useful.
We can delay hard destructions until later
prolog freeze





## AC

AC unify is exact opposite of AC KB?
Multiset something


matrices of subsitutions. "multply" is subst composition, add is multiset combine?


## Boolean
That recent typechekcing paper flix https://dl.acm.org/doi/10.1145/3485487 Relational nullable types with Boolean unification
https://www.youtube.com/watch?v=BPW-92b7j7A&ab_channel=ACMSIGPLAN fast and efficient boolean unification for hindley milner

boolean unification the sotry so far https://www.cs.rice.edu/~javaplt/411/23-spring/NewReadings/Unfication%20Theory/Boolean-unification---The-story-so-far_1989_Journal-of-Symbolic-Computation.pdf


Reduce to f(x) = g(x) becomes f(x) + g(x) = 0.

loweneim and booles method



## Generic Narrowing

System G
{e[u]} ==> {l = u, e[r]}
lazy paramdoulation is bruteforce
u can't be a variable
It avoids needing unification to perform narrowing.

unification modulo egraphs


Fey and Hullot
narrow



## Functional Logic Programming

https://www.curry-language.org/docs/tutorial/tutorial.pdf curry a ttutorial introduction

https://www.michaelhanus.de/papers/GanzingerFestschrift.pdf from narrowing to curry


functional logic programming
https://dl.acm.org/doi/pdf/10.1145/1721654.1721675 review article

inductively sequential
definition trees

weakly orthogonal. There is non trivial overlap but all critical pairs are immiedately good.

flattening. append(x,y) = z --> append(x,y,z) Different orderings of flattenings make different eval strategies
`t = e` :- 

flp minikanren?
curry
verse

lambda flp?

narrowing
residuization\

functional logic programming.
ground flp proplog
https://web.cecs.pdx.edu/~antoy/homepage/publications/narrowing/paper.pdf 

https://simon.peytonjones.org/assets/pdfs/verse-conf.pdf verse

https://arxiv.org/pdf/2405.10801  The Relational Machine Calculus

https://maude.lcc.uma.es/manual271/maude-manualch16.html Narrwoing maude
https://maude.lcc.uma.es/maude31-manual-html/maude-manualch15.html
[77]   Joseph Goguen and José Meseguer. Eqlog: Equality, types and generic modules for logic programming. 
]   Michael Hanus. The integration of functions into logic programming: From theory to practice
 José Meseguer. Multiparadigm logic programming
] James R. Slagle. Automated Theorem-Proving for Theories with Simplifiers
Commutativity, and Associativity


 https://dl.acm.org/doi/abs/10.1145/1016850.1016865  Implementing functional logic languages using multiple threads and stores

 https://pdxscholar.library.pdx.edu/cgi/viewcontent.cgi?article=7501&context=open_access_etds  Implementing a Functional Logic Programming Language via the Fair Scheme

What about a functional logic programming approach to marshall? We had these taylor series approximations. It's kind of intuitiopnsitc in some sense. Maybe bring in unification modulo polynomials? Modulo polynomial inequalities?

sergio antoy https://web.cecs.pdx.edu/~antoy/
michael hanus

https://web.cecs.pdx.edu/~antoy/homepage/theses/A_Peters.pdf  The Basic Scheme for the Evaluation of Functional Logic Programs . relatively simple ocaml

cloning
needed narrowing - https://www.informatik.uni-kiel.de/~mh/papers/JACM00.pdf
bubbling
pull tabbing

goedel

maybe kics is the way to go.


Could be fun to mix evaluation a la unfold with narrowing via rules. That's the idea right?
Or evaluation via reify + narrow.


In [ ]:
def rewrite_search(t, rules):
    seen = {}


In [ ]:
from kdrag.all import *
import kdrag.rewrite as rw

#def ground(vs, t): / is_value?
# 


# could do a rewriting
#  search similarly you know.
def flp(vs, t, rules):
    rules = [
        rule if isinstance(rule, rw.RewriteRule) else rw.rewrite_of_expr(rule)
        for rule in rules
    ]
    subst = {}
    todo = [(vs, t, subst)] # could have mutiple t?
    while todo:
        vs,t,subst = todo.pop()
        progress = False
        for rule in rules:
            rule = rule.freshen()
            narrows = rw.all_narrows(vs + rule.vs, t, rule.lhs, rule.rhs)
            if len(narrows) != 0: # no. I don't want this just because 
                progress = True
            for t1, subst1 in rw.all_narrows(vs, t, rule.lhs, rule.rhs):
                # combine subst and subst1
                # reduce vs maybe
                # maybe yield here if reach some condition
                todo.append((vs, t1, subst | subst1))
        if not progress:
            yield vs, t, subst

import kdrag.theories.nat as nat
rules = [
    nat.add_zero,
    nat.add_succ,
]


COuld be fun to table too. Variant tabling.

In [ ]:
from kdrag.all import *
def prolog(vs0 : list[smt.ExprRef], goals : smt.BoolRef, rules : list[rw.RewriteRule]):
    rules = [
        rule if isinstance(rule, rw.Rule) else rw.rule_of_expr(rule)
        for rule in reversed(rules)
    ]
    todo = [(vs0, goals, {})]
    while todo:
        vs, goals, subst = todo.pop()
        if len(goals) == 0:
            yield vs, {k : t for k,t in subst.items() if any(k.eq(v) for v in vs0)}
            continue
        else:
            goal = goals.pop()
            if smt.is_true(goal):
                todo.append((vs, goals, subst))
            elif smt.is_false(goal):
                continue
            elif smt.is_and(goal):
                goals.extend(reversed(goal.children()))
                todo.append((vs, goals, subst))
            elif smt.is_or(goal):
                for child in goal.children():
                    newgoals = goals + [child]
                    todo.append((vs, newgoals, subst))
            else:
                for rule in rules:
                    rule = rule.freshen()
                    vs1 = vs + rule.vs
                    subst1 = kd.utils.unify(vs1, rule.conc, goal)
                    if subst1 is None:
                        continue
                    else:
                        newgoals = goals + [smt.substitute(rule.hyp, *subst1.items())]
                        newsubst = {**{k : smt.substitute(v, *subst1.items()) for k,v in subst.items()}, **subst1}
                        newvs = list(set(vs1) - set(subst1.keys()))
                        todo.append((newvs, newgoals, newsubst))

import kdrag.theories.list as list_
import kdrag.theories.nat as nat
ListInt = list_.List(smt.IntSort())
x,y,z = smt.Ints("x y z")

x,y,z = smt.Consts("x y z", nat.Nat)
plus = smt.Function("plus", nat.Nat, nat.Nat, nat.Nat, smt.BoolSort())
rules = [
kd.QForAll([y], plus(nat.Z, y, y)),
kd.QForAll([x,y,z], smt.Implies(
    plus(x, y, z), 
    plus(nat.S(x), y, nat.S(z))
))
]
print(list(prolog([x,y], [plus(x, y, nat.S(nat.Z))], rules)))
print(list(prolog([x,y], [plus(x, y, nat.S(nat.Z))], rules)))
print(list(prolog([x,y], [plus(nat.Z, nat.Z, nat.S(nat.Z))], rules)))
import itertools
print(list(itertools.islice(prolog([x,y,z], [plus(x, y, nat.S(z))], rules), 4)))


[([], {y: S(Z), x: Z}), ([], {x: S(Z), y: Z})]
[([], {y: S(Z), x: Z}), ([], {x: S(Z), y: Z})]
[]
[([z], {y: S(z), x: Z}), ([y], {x: S(Z), z: y}), ([y], {x: S(S(Z)), z: S(y)}), ([y], {x: S(S(S(Z))), z: S(S(y))})]


In [ ]:
def datalog(facts, rules):
    db = {fact.get_id() : fact for fact in facts}
    rules = 


In [ ]:
%%file /tmp/hello.curry

{-# LANGUAGE NoImplicitPrelude #-}
data Nat = Z | S Nat
data MyBool = False | True
-- Addition on natural numbers.
add :: Nat -> Nat -> Nat
add Z n = n
add (S m) n = S (add m n)
-- Less-or-equal predicate on natural numbers.
leq :: Nat -> Nat -> MyBool
leq Z _ = True
leq (S _) Z = False
leq (S x) (S y) = leq x y


just_doit = 3



Overwriting /tmp/hello.curry


In [2]:
!kics2 :eval just_doit :quit

/bin/bash: line 1: kics2: command not found


In [6]:
%%file /tmp/hello.curry
main = IO.println "hello world"

Overwriting /tmp/hello.curry


In [8]:
%%file /tmp/hello.curry
main = print "hello world"

Overwriting /tmp/hello.curry


In [ ]:
! runcurry /tmp/hello.curry

cypm install runcurry